In [ ]:
%%javascript
//
//
// DO NOT EXECUTE THIS IN THE NOTEBOOK, USE prepare_javascript 
//
//
var ws = null
var isopen = false
var intervalID = null

// create websocket instance
var port = 50107
var uri = '/testsocket/'
var loc = document.location, new_uri, url;
if (loc.protocol === "https:") {
    new_uri = "wss:";
} else {
    new_uri = "ws:";
}

if (document.location.hostname.includes("localhost")){
    new_uri += '//' + document.location.hostname + ':' + port + uri;
    url = new_uri;
}
else {
    new_uri += '//' + document.location.hostname + document.location.pathname + uri;
    url = new_uri.replace("notebooks", "proxy/"+port);
}
console.log(url);
ws = new WebSocket(url);

ws.binaryType = "arraybuffer";

// Handle incoming websocket message callback
ws.onmessage = function(evt) {
    console.log("WebSocket Message Received: " + evt.data)
};

// Close Websocket callback
ws.onclose = function(event) {
    ws = null
    isopen = false
    console.log("onclose intervalID = ",intervalID)
    clearInterval(intervalID);
    console.log("***WebSocket Connection Closed***");
};

// Open Websocket callback
ws.onopen = function(evt) {
    isopen = true
    console.log("***WebSocket Connection Opened***");
    intervalID = setInterval(function(){if (ws.readyState === WebSocket.OPEN) {ws.send("Hello World");console.log("Sent 'Hello World' to websocket");}else{console.log("Websocket is not OPEN")}}, 5000);
    console.log("open intervalID = ",intervalID)
    setTimeout(function(){ ws.close();console.log("Closing Websocket"); }, 30000);

};

ws.onerror = function(event) {
    console.error("WebSocket error observed:", event);
};


In [ ]:
%%javascript
//
//
// DO NOT EXECUTE THIS IN THE NOTEBOOK, USE prepare_javascript 
//
//
var analyser;
SAMPLE_RATE = 16000;
SAMPLE_SIZE = 16;
var playButton = document.getElementById('playbutton');
// Hook up the play/pause state to the microphone context
var context = new AudioContext();
var socket = null;
var isopen = false;
var intervalID = null;
var audioPromise = null;
var scriptNode = null;
var sourceNode = null;
playButton.addEventListener('pause', context.suspend.bind(context));
playButton.addEventListener('play', context.resume.bind(context));
playButton.addEventListener('click', function startRecording() {
    if(audioPromise === null){
        audioPromise = navigator.mediaDevices.getUserMedia({
          audio: {
            echoCancellation: true,
            channelCount: 1,
            sampleRate: {
              ideal: SAMPLE_RATE
            },
            sampleSize: SAMPLE_SIZE
          }
        });

        audioPromise.then(function(micStream) {
          var microphone = context.createMediaStreamSource(micStream);
          analyser = context.createAnalyser();
          microphone.connect(analyser);
        }).catch(console.log.bind(console));
    }
    console.log('click');
    if(socket === null){
        console.log('creating WS');
        initWebsocket(audioPromise);
    }
    else{
        console.log('closing WS');
        socket.close();
    }
});
function closeWebsocket() {
    console.log("onclose intervalID = ",intervalID);
    scriptNode.disconnect();
    if (sourceNode) sourceNode.disconnect();
    clearInterval(intervalID);
    console.log("***WebSocket Connection Closed***");
    socket = null;
    isopen = false;
}

/**
* Hook up event handlers to create / destroy websockets, and audio nodes to
* transmit audio bytes through it.
*/
function initWebsocket(audioPromise) {
    // Create a node that sends raw bytes across the websocket
    scriptNode = context.createScriptProcessor(4096, 1, 1);
    // Need the maximum value for 16-bit signed samples, to convert from float.
    const MAX_INT = Math.pow(2, 16 - 1) - 1;
    scriptNode.addEventListener('audioprocess', function(e) {
        var floatSamples = e.inputBuffer.getChannelData(0);
        // The samples are floats in range [-1, 1]. Convert to 16-bit signed
        // integer.
        socket.send(Int16Array.from(floatSamples.map(function(n) {
            return n * MAX_INT;
        })));
    });

    function newWebsocket() {
        // create websocket instance
        var port = 50107
        var uri = '/ws'
        var loc = document.location, new_uri, url;

        if (loc.protocol === "https:") {
            new_uri = "wss:";
        } else {
            new_uri = "ws:";
        }
        if (document.location.hostname.includes("localhost")){
            new_uri += '//' + document.location.hostname + ':' + port + uri;
            url = new_uri;
        }
        else {
            new_uri += '//' + document.location.hostname + document.location.pathname + uri;
            url = new_uri.replace("notebooks", "proxy/"+port);
        }
        //ws = new WebSocket(url);
        var websocketPromise = new Promise(function(resolve, reject) {
            var socket = new WebSocket(url);
            socket.binaryType = "arraybuffer";
            socket.onopen = function(evt) {
                isopen = true
                console.log("***WebSocket Connection Opened***");
            };

            // Close Websocket callback
            socket.onclose = function(evt) {
                closeWebsocket();
                playButton.dispatchEvent(new Event('pause'));
            };
            socket.onerror = function(event) {
                console.error("WebSocket error observed:", event);
                playButton.dispatchEvent(new Event('pause'));
            };
            socket.onmessage = function(evt) {
                console.log("WebSocket Message Received: " + evt.data)
            };
            console.log(url);
            socket.addEventListener('open', resolve);
            socket.addEventListener('error', reject);
        });
        Promise.all([audioPromise, websocketPromise]).then(function(values) {
            var micStream = values[0];
            socket = values[1].target;
            console.log("reaches here!!");

            function startByteStream(e) {
                // Hook up the scriptNode to the mic
                console.log("reaches here also!!");
                sourceNode = context.createMediaStreamSource(micStream);
                sourceNode.connect(scriptNode);
                scriptNode.connect(context.destination);
            }

            // Send the initial configuration message. When the server acknowledges
            // it, start streaming the audio bytes to the server and listening for
            // transcriptions.
            socket.addEventListener('message', function(e) {
                socket.addEventListener('message', onTranscription);
                startByteStream(e);
            }, {once: true});
            console.log('sending sample rate');
            socket.send(JSON.stringify({sampleRate: context.sampleRate}));
            console.log('sample rate sent');

        }).catch(console.log.bind(console));



        // Handle incoming websocket message callback
        /*
        ws.binaryType = "arraybuffer";
        ws.onmessage = function(evt) {
            console.log("WebSocket Message Received: " + evt.data)
        };

        // Open Websocket callback
        ws.onopen = function(evt) {
            isopen = true
            console.log("***WebSocket Connection Opened***");
            intervalID = setInterval(function(){if (ws.readyState === WebSocket.OPEN) {ws.send("Hello World");console.log("Sent 'Hello World' to websocket");}else{console.log("Websocket is not OPEN"); closeWebsocket(); clearInterval(intervalID);}}, 5000);
            console.log("open intervalID = ",intervalID)
            //setTimeout(function(){ ws.close();console.log("Closing Websocket"); }, 30000);

        };*/
    }
    newWebsocket();
}

# Python

In [ ]:
from IPython.display import Javascript
import urllib.parse
import json

# https://mybinder.org/v2/gh/jcoady/notebook/master
import os
import time
from threading import Thread

from jupyter_core.paths import jupyter_data_dir
import notebook
import IPython
from IPython.display import display, Javascript

import tornado.httpserver
import tornado.websocket
import tornado.ioloop
import tornado.web
import socket
import json
import asyncio
import logging

def find_free_port():
    s = socket.socket()
    s.bind(('',0)) # find an available port
    return s.getsockname()[1]

__SOCKET_PORT = find_free_port()

wsConnected = False

class WSHandler(tornado.websocket.WebSocketHandler):
    def open(self):
        global wsConnected
        wsConnected = True
        print("Websocket Open")

    def on_message(self, message):
        #ws_queue.put(message)
        pass

    def on_close(self):
        print("Close Websocket")
        self.stop_tornado()

    def stop_tornado(self):
        ioloop = tornado.ioloop.IOLoop.instance()
        ioloop.add_callback(ioloop.stop)

    def check_origin(self, origin):
        return True

def start_server():
    asyncio.set_event_loop(asyncio.new_event_loop())
    application = tornado.web.Application([(r'/ws', WSHandler),])
    http_server = tornado.httpserver.HTTPServer(application)
    http_server.listen(__SOCKET_PORT)
    Log = logging.getLogger('tornado.access')
    level = logging.getLevelName('WARN')
    Log.setLevel(level)
    tornado.ioloop.IOLoop.instance().start()


def prepare_javascript(js_strings): # Prepare the first javascript string found.
    magic_string = "%%javascript" 
    for s in js_strings:
        if s is not None and magic_string == s[:len(magic_string)]:
            js_string = s\
            .replace("%%javascript","")\
            .replace('\\n','\n').replace('\\"','"')\
            .replace("\\'","'")
            js_string = f'<script type="text/Javascript">\n{js_string}\n</script>'
        return js_string
    return None

Javascript("""
var source_code = Jupyter.notebook.get_cells().map((cell) => {
    if (cell.cell_type == "code") {
        var source = cell.code_mirror.getValue();
        if (!source.startsWith("%%javascript")) {
            return "";
        }
        else{
            return source;
        }
    }
});

source_code = JSON.stringify(source_code);
source_code = encodeURI(source_code);
source_code = '_js_source_code="' + source_code + '"';
var kernel = IPython.notebook.kernel;
var t = kernel.execute(source_code);
""")

In [ ]:
from IPython.display import HTML
import time

t = Thread(target=start_server, args=())
t.start()

time.sleep(2)

input_form = """
<div>
    <input id="playbutton" type="button" value="PLAY">
    <div id="transcript">
        <div></div>
    </div>
</div>
"""

js_strings = json.loads(urllib.parse.unquote(_js_source_code))
javascript = prepare_javascript(js_strings)
HTML(input_form + javascript)